In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['axes.unicode_minus'] = False
import warnings
warnings.filterwarnings("ignore")
df = pd.read_csv('RL-10.csv')
df

In [ ]:
from sklearn.preprocessing import LabelEncoder
# 将 y_train 的标签重新编码为从0开始的整数
label_encoder = LabelEncoder()
df['NSP(label)_encoded'] = label_encoder.fit_transform(df['NSP(label)'])

# 输出原始标签和编码后的标签
for original, encoded in zip(df['NSP(label)'], df['NSP(label)_encoded']):
    print(f"Original: {original}, Encoded: {encoded}")

In [ ]:
# 划分特征和目标变量
X = df.drop(['label', 'ID'], axis=1)
y = df['label']
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42, stratify=df['label'])
df.head()

In [ ]:
def split_data(X, y):
    # 先划分训练集（60%），剩下40%再均分为测试集和验证集（各20%）
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
    return X_train, X_test, X_val, y_train, y_test, y_val
df.head()
# 假设 X, y 已经定义
X_train, X_test, X_val, y_train, y_test, y_val = split_data(X, y)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# 多分类XGBoost模型参数
params_xgb = {
    'learning_rate': 0.02,              # 学习率，控制每一步的步长
    'booster': 'gbtree',                # 使用梯度提升树
    'objective': 'multi:softmax',       # 损失函数，用于多分类的softmax
    'num_class': 3,                     # 分类类别数量，需根据具体的多分类任务调整
    'max_leaves': 127,                  # 每棵树的叶子节点数量
    'verbosity': 1,                     # 控制输出信息
    'seed': 42,                         # 随机种子
    'nthread': -1,                      # 使用所有可用的CPU核心
    'colsample_bytree': 0.6,            # 每棵树随机选择的特征比例
    'subsample': 0.7,                   # 每次迭代时随机选择的样本比例
    'eval_metric': 'mlogloss'           # 使用多分类对数损失作为评价指标
}

# 初始化XGBoost多分类模型
model_xgb = xgb.XGBClassifier(**params_xgb)

# 定义参数网格，用于网格搜索
param_grid = {
    'n_estimators': [100, 200, 300],    # 树的数量
    'max_depth': [3, 4, 5, 6],          # 树的深度
    'learning_rate': [0.01, 0.02, 0.05, 0.1]  # 学习率
}

# 使用GridSearchCV进行网格搜索和k折交叉验证
grid_search = GridSearchCV(
    estimator=model_xgb,
    param_grid=param_grid,
    scoring='neg_log_loss',  # 评价指标为负对数损失
    cv=5,                    # 5折交叉验证
    n_jobs=-1,               # 并行计算
    verbose=1                # 输出详细进度信息
)

# 训练模型
grid_search.fit(X_train, y_train)

# 输出最优参数
print("Best parameters found: ", grid_search.best_params_)
print("Best Log Loss score: ", -grid_search.best_score_)

# 使用最优参数训练模型
best_model = grid_search.best_estimator_

In [ ]:
# 使用模型在测试集上进行预测
y_pred= best_model .predict(X_test)
from sklearn.metrics import classification_report
# 输出模型报告， 查看评价指标
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
# 输出混淆矩阵
conf_matrix = confusion_matrix(y_test, y_pred)
# 绘制热力图
plt.figure(figsize=(10, 7), dpi = 1200)
sns.heatmap(conf_matrix, annot=True, annot_kws={'size':15}, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.title('XGBoost Confusion matrix heat map', fontsize=15)
plt.savefig('XGBoost Confusion matrix heat map.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    conf_matrix = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(7, 5), dpi=1200)
    sns.heatmap(conf_matrix, annot=True, annot_kws={'size':15}, fmt='d', cmap='Blues')
    plt.xlabel('Predicted Label', fontsize=12)
    plt.ylabel('True Label', fontsize=12)
    plt.title(title, fontsize=15)
    plt.savefig(save_path, format='pdf', bbox_inches='tight')
    plt.show()
    print(classification_report(y_true, y_pred))

# 训练集
y_pred_train = best_model.predict(X_train)
plot_confusion_matrix(y_train, y_pred_train, 'XGBoost Confusion matrix heat map (Train)', 'XGBoost_Confusion_matrix_train.pdf')

# 测试集
y_pred_test = best_model.predict(X_test)
plot_confusion_matrix(y_test, y_pred_test, 'XGBoost Confusion matrix heat map (Test)', 'XGBoost_Confusion_matrix_test.pdf')

# 验证集
y_pred_val = best_model.predict(X_val)
plot_confusion_matrix(y_val, y_pred_val, 'XGBoost Confusion matrix heat map (Validation)', 'XGBoost_Confusion_matrix_val.pdf')

In [ ]:
from sklearn import metrics
from sklearn.preprocessing import label_binarize

# 预测并计算概率
ytest_proba_xgb = best_model.predict_proba(X_test)

# 将y标签转换成one-hot形式
ytest_one_xgb = label_binarize(y_test, classes=[0, 1, 2])

# 宏平均法计算AUC
xgb_AUC = {}
xgb_FPR = {}
xgb_TPR = {}

for i in range(ytest_one_xgb.shape[1]):
    xgb_FPR[i], xgb_TPR[i], thresholds = metrics.roc_curve(ytest_one_xgb[:, i], ytest_proba_xgb[:, i])
    xgb_AUC[i] = metrics.auc(xgb_FPR[i], xgb_TPR[i])
print(xgb_AUC)

# 合并所有的FPR并排序去重
xgb_FPR_final = np.unique(np.concatenate([xgb_FPR[i] for i in range(ytest_one_xgb.shape[1])]))

# 计算宏平均TPR
xgb_TPR_all = np.zeros_like(xgb_FPR_final)
for i in range(ytest_one_xgb.shape[1]):
    xgb_TPR_all += np.interp(xgb_FPR_final, xgb_FPR[i], xgb_TPR[i])
xgb_TPR_final = xgb_TPR_all / ytest_one_xgb.shape[1]

# 计算最终的宏平均AUC
xgb_AUC_final = metrics.auc(xgb_FPR_final, xgb_TPR_final)
AUC_final_xgb = xgb_AUC_final  # 最终AUC

print(f"Macro Average AUC with XGBoost: {AUC_final_xgb}")

In [ ]:
plt.figure(figsize=(7, 7), dpi=1200)
# 使用不同的颜色和线型绘制不同类别的ROC曲线
plt.plot(xgb_FPR[0], xgb_TPR[0], color='#1f77b4', linestyle='-', label='Class 0 ROC  AUC={:.4f}'.format(xgb_AUC[0]), lw=2)
plt.plot(xgb_FPR[1], xgb_TPR[1], color='#ff7f0e', linestyle='-', label='Class 1 ROC  AUC={:.4f}'.format(xgb_AUC[1]), lw=2)
plt.plot(xgb_FPR[2], xgb_TPR[2], color='#2ca02c', linestyle='-', label='Class 2 ROC  AUC={:.4f}'.format(xgb_AUC[2]), lw=2)
# 宏平均ROC曲线
plt.plot(xgb_FPR_final, xgb_TPR_final, color='#000000', linestyle='-', label='Macro Average ROC  AUC={:.4f}'.format(xgb_AUC_final), lw=3)
# 45度参考线
plt.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=2, label='45 Degree Reference Line')
# 设置标签、标题和图例
plt.xlabel('False Positive Rate (FPR)', fontsize=15)
plt.ylabel('True Positive Rate (TPR)', fontsize=15)
plt.title('XGBoost Classification ROC Curves and AUC', fontsize=18)
plt.grid(linestyle='--', alpha=0.7)
plt.legend(loc='lower right', framealpha=0.9, fontsize=12)
plt.savefig('XGBoost_optimized.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
from sklearn import metrics
from sklearn.preprocessing import label_binarize

def plot_multiclass_roc(X, y, model, classes, title, save_path):
    # 预测概率
    y_proba = model.predict_proba(X)
    # one-hot编码
    y_onehot = label_binarize(y, classes=classes)
    # 每个类别的ROC
    auc_dict = {}
    fpr_dict = {}
    tpr_dict = {}
    for i in range(y_onehot.shape[1]):
        fpr_dict[i], tpr_dict[i], _ = metrics.roc_curve(y_onehot[:, i], y_proba[:, i])
        auc_dict[i] = metrics.auc(fpr_dict[i], tpr_dict[i])
    # 宏平均
    fpr_all = np.unique(np.concatenate([fpr_dict[i] for i in range(y_onehot.shape[1])]))
    tpr_all = np.zeros_like(fpr_all)
    for i in range(y_onehot.shape[1]):
        tpr_all += np.interp(fpr_all, fpr_dict[i], tpr_dict[i])
    tpr_final = tpr_all / y_onehot.shape[1]
    auc_final = metrics.auc(fpr_all, tpr_final)
    # 绘图
    plt.figure(figsize=(6, 6), dpi=1200)
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    for i in range(y_onehot.shape[1]):
        plt.plot(fpr_dict[i], tpr_dict[i], color=colors[i], linestyle='-',
                 label=f'Class {i} ROC  AUC={auc_dict[i]:.4f}', lw=2)
    plt.plot(fpr_all, tpr_final, color='#000000', linestyle='-', 
             label=f'Macro Average ROC  AUC={auc_final:.4f}', lw=3)
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=2, label='45 Degree Reference Line')
    plt.xlabel('False Positive Rate (FPR)', fontsize=15)
    plt.ylabel('True Positive Rate (TPR)', fontsize=15)
    plt.title(title, fontsize=18)
    plt.grid(False)
    plt.legend(loc='lower right', framealpha=0.9, fontsize=12)
    plt.savefig(save_path, format='pdf', bbox_inches='tight')
    plt.show()

# 假设类别为[0,1,2]
classes = [0, 1, 2]

# 绘制训练集ROC
plot_multiclass_roc(X_train, y_train, best_model, classes, 'XGBoost Train ROC Curves and AUC', 'XGBoost_train_roc.pdf')

# 绘制测试集ROC
plot_multiclass_roc(X_test, y_test, best_model, classes, 'XGBoost Test ROC Curves and AUC', 'XGBoost_test_roc.pdf')

# 绘制验证集ROC（如果有验证集）
plot_multiclass_roc(X_val, y_val, best_model, classes, 'XGBoost Validation ROC Curves and AUC', 'XGBoost_val_roc.pdf')


In [ ]:
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import numpy as np

def plot_multiclass_calibration(X, y, model, classes, title, save_path):
    plt.figure(figsize=(7, 7), dpi=1200)
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    y_proba = model.predict_proba(X)
    for i, color in zip(classes, colors):
        prob_true, prob_pred = calibration_curve((y == i).astype(int), y_proba[:, i], n_bins=10)
        plt.plot(prob_pred, prob_true, marker='o', color=color, label=f'Class {i}')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfectly calibrated')
    plt.xlabel('Mean predicted probability', fontsize=15)
    plt.ylabel('Fraction of positives', fontsize=15)
    plt.title(title, fontsize=18)
    plt.legend(loc='best', fontsize=12)
    plt.grid(False)
    plt.savefig(save_path, format='pdf', bbox_inches='tight')
    plt.show()

classes = [0, 1, 2]

# 训练集
plot_multiclass_calibration(X_train, y_train, best_model, classes, 'XGBoost Calibration Curve (Train)', 'XGBoost_calibration_train.pdf')
# 测试集
plot_multiclass_calibration(X_test, y_test, best_model, classes, 'XGBoost Calibration Curve (Test)', 'XGBoost_calibration_test.pdf')
# 验证集
plot_multiclass_calibration(X_val, y_val, best_model, classes, 'XGBoost Calibration Curve (Validation)', 'XGBoost_calibration_val.pdf')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_dca_curve_multiclass(X, y, model, classes, title, save_path):
    plt.figure(figsize=(7, 7), dpi=1200)
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    thresholds = np.linspace(0.01, 0.99, 100)
    for idx, color in zip(classes, colors):
        y_proba = model.predict_proba(X)[:, idx]
        y_true = (y == idx).astype(int)
        N = len(y_true)
        net_benefit = []
        for pt in thresholds:
            tp = np.sum((y_proba >= pt) & (y_true == 1))
            fp = np.sum((y_proba >= pt) & (y_true == 0))
            nb = (tp / N) - (fp / N) * (pt / (1 - pt))
            net_benefit.append(nb)
        plt.plot(thresholds, net_benefit, color=color, lw=2, label=f'Class {idx}')
    # 参考线
    treat_none = np.zeros_like(thresholds)
    plt.plot(thresholds, treat_none, color='black', linestyle=':', label='Treat None')
    plt.xlabel('Threshold Probability', fontsize=15)
    plt.ylabel('Net Benefit', fontsize=15)
    plt.title(title, fontsize=18)
    plt.legend(fontsize=12)
    plt.grid(linestyle='--', alpha=0.7)
    plt.savefig(save_path, format='pdf', bbox_inches='tight')
    plt.show()

classes = [0, 1, 2]

# 训练集
plot_dca_curve_multiclass(X_train, y_train, best_model, classes, 'DCA Curve (Train)', 'DCA_train.pdf')
# 测试集
plot_dca_curve_multiclass(X_test, y_test, best_model, classes, 'DCA Curve (Test)', 'DCA_test.pdf')
# 验证集
plot_dca_curve_multiclass(X_val, y_val, best_model, classes, 'DCA Curve (Validation)', 'DCA_val.pdf')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
plt.rcParams['font.family'] = 'Times New Roman'
# 修改DCA计算函数以支持多分类
def calculate_net_benefit(thresh_group, y_pred_score, y_label, n_classes):
    net_benefit = np.zeros((n_classes, len(thresh_group)))  # 为每个类别创建数组
    for c in range(n_classes):
        for i, thresh in enumerate(thresh_group):
            y_pred_label = (y_pred_score[:, c] > thresh).astype(int)  # 对每个类别计算预测标签
            tn, fp, fn, tp = confusion_matrix(y_label == c, y_pred_label).ravel()
            n = len(y_label)
            net_benefit[c, i] = (tp / n) - (fp / n) * (thresh / (1 - thresh))
    return net_benefit

# 更新绘图函数来处理多分类的DCA曲线
def plot_DCA(ax, thresh_group, net_benefit, label, color, n_classes):
    for c in range(n_classes):
        ax.plot(thresh_group, net_benefit[c], color=color, label=f'{label} Class {c}')

# 计算Treat all的净收益
def calculate_net_benefit_all(thresh_group, y_label, n_classes):
    net_benefit_all = np.zeros((n_classes, len(thresh_group)))
    for c in range(n_classes):
        tn, fp, fn, tp = confusion_matrix(y_label == c, y_label == c).ravel()
        total = tp + tn
        for i, thresh in enumerate(thresh_group):
            net_benefit_all[c, i] = (tp / total) - (tn / total) * (thresh / (1 - thresh))
    return net_benefit_all

# 绘制所有模型的DCA曲线
def plot_all_DCA(models, X, y, thresh_group, n_classes):
    fig, ax = plt.subplots(figsize=(8, 6))

    # 计算Treat all的净收益
    net_benefit_all = calculate_net_benefit_all(thresh_group, y, n_classes)
    
    # 绘制每个模型的DCA曲线
    for model_name, (model, color) in models.items():
        model.fit(X, y)
        y_pred_score = model.predict_proba(X)  # 获取每个类别的概率
        net_benefit = calculate_net_benefit(thresh_group, y_pred_score, y, n_classes)
        plot_DCA(ax, thresh_group, net_benefit, model_name, color, n_classes)

    # 添加 Treat None 和 Treat All 线
    ax.plot(thresh_group, np.zeros_like(thresh_group), color='black', linestyle=':', label='Treat None')
    ax.plot(thresh_group, net_benefit_all.mean(axis=0), color='black', label='Treat All')

    # 图表配置和美化
    ax.set_xlim(0, 1)
    ax.set_ylim(-0.5, 0.8)
    ax.set_xlabel('Risk Threshold', fontdict={'family': 'Times New Roman', 'fontsize': 15})
    ax.set_ylabel('Net Benefit', fontdict={'family': 'Times New Roman', 'fontsize': 15})
    ax.grid(False)
    ax.legend(loc='lower left')
    plt.title('DCA Curves Test set', fontsize=16)
    plt.tight_layout()
    plt.savefig('DCA_Curves_Test.pdf', dpi=1200)
    plt.show()

# 修复模型字典中的 RandomForest 参数
models = {
    'XGBoost': (XGBClassifier(), 'blue')
}

# 设定阈值范围
thresh_group = np.linspace(0, 1, 100)

# 调用绘图函数进行DCA曲线绘制
# 这里你可以根据实际数据使用 X_train, y_train 或 X_test, y_test
plot_all_DCA(models, X_test, y_test, thresh_group, n_classes=3)
#plot_all_DCA(models, X_train, y_train, thresh_group, n_classes=3)
#plot_all_DCA(models, X_exvad, y_exvad, thresh_group, n_classes=3)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

plt.rcParams['font.family'] = 'Times New Roman'

# 修改 DCA 计算函数以支持多分类
def calculate_net_benefit(thresh_group, y_pred_score, y_label, n_classes):
    net_benefit = np.zeros((n_classes, len(thresh_group)))
    for c in range(n_classes):
        for i, thresh in enumerate(thresh_group):
            y_pred_label = (y_pred_score[:, c] > thresh).astype(int)
            tn, fp, fn, tp = confusion_matrix(y_label == c, y_pred_label).ravel()
            n = len(y_label)
            net_benefit[c, i] = (tp / n) - (fp / n) * (thresh / (1 - thresh))
    return net_benefit

# 更新绘图函数来处理多分类的 DCA 曲线
def plot_DCA(ax, thresh_group, net_benefit, label, color, n_classes):
    for c in range(n_classes):
        ax.plot(thresh_group, net_benefit[c], color=color, label=f'{label} Class {c}')

# 计算 Treat All 的净收益（修正逻辑）
def calculate_net_benefit_all(thresh_group, y_label, n_classes):
    net_benefit_all = np.zeros((n_classes, len(thresh_group)))
    for c in range(n_classes):
        count_i = np.sum(y_label == c)  # 实际属于类别 c 的样本数
        N = len(y_label)  # 总样本数
        for i, thresh in enumerate(thresh_group):
            nb = (count_i / N) - ((N - count_i) / N) * (thresh / (1 - thresh))
            net_benefit_all[c, i] = nb
    return net_benefit_all

# 绘制所有模型的 DCA 曲线
def plot_all_DCA(models, X, y, thresh_group, n_classes):
    fig, ax = plt.subplots(figsize=(8, 6))

    # 计算 Treat All 的净收益
    net_benefit_all = calculate_net_benefit_all(thresh_group, y, n_classes)

    # 绘制每个模型的 DCA 曲线
    for model_name, (model, color) in models.items():
        model.fit(X, y)
        y_pred_score = model.predict_proba(X)  # 获取每个类别的概率
        net_benefit = calculate_net_benefit(thresh_group, y_pred_score, y, n_classes)
        plot_DCA(ax, thresh_group, net_benefit, model_name, color, n_classes)

    # 添加 Treat None 和 Treat All 线
    ax.plot(thresh_group, np.zeros_like(thresh_group), color='black', linestyle=':', label='Treat None')
    for c in range(n_classes):
        ax.plot(thresh_group, net_benefit_all[c], color='gray', linestyle='--', label=f'Treat All Class {c}' if c == 0 else None)

    # 图表配置和美化
    ax.set_xlim(0, 1)
    ax.set_ylim(-0.5, 0.8)
    ax.set_xlabel('Risk Threshold', fontdict={'family': 'Times New Roman', 'fontsize': 15})
    ax.set_ylabel('Net Benefit', fontdict={'family': 'Times New Roman', 'fontsize': 15})
    ax.grid(False)
    ax.legend(loc='lower left')
    plt.title('DCA Curves for XGBoost Model in Test set', fontsize=16)
    plt.tight_layout()
    plt.savefig('DCA_Curves_Test.pdf', dpi=1200)
    plt.show()

# 修复模型字典（仅保留 XGBoost）
models = {
    'XGBoost': (XGBClassifier(), 'blue'),
}

# 设定阈值范围
thresh_group = np.linspace(0, 1, 100)

# 调用绘图函数进行 DCA 曲线绘制
#plot_all_DCA(models, X_test, y_test, thresh_group, n_classes=3)
#plot_all_DCA(models, X_train, y_train, thresh_group, n_classes=3)
plot_all_DCA(models, X_val, y_val, thresh_group, n_classes=3)

In [ ]:
def plot_dca_curve_multiclass(X, y, model, classes, title, save_path):
    plt.figure(figsize=(7, 7), dpi=1200)
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    thresholds = np.linspace(0.01, 0.99, 100)
    
    for idx, color in zip(classes, colors):
        y_proba = model.predict_proba(X)[:, idx]
        y_true = (y == idx).astype(int)
        N = len(y_true)
        
        # 计算模型预测的净收益
        net_benefit = []
        for pt in thresholds:
            tp = np.sum((y_proba >= pt) & (y_true == 1))
            fp = np.sum((y_proba >= pt) & (y_true == 0))
            nb = (tp / N) - (fp / N) * (pt / (1 - pt))
            net_benefit.append(nb)
        plt.plot(thresholds, net_benefit, color=color, lw=2, label=f'Class {idx}')
        
        # 计算Treat All线
        prevalence = y_true.mean()  # 等价于 tp_all/N
        fp_all = len(y_true) - y_true.sum()
        nb_treat_all = [prevalence - (fp_all/N)*(pt/(1-pt)) for pt in thresholds]
        plt.plot(thresholds, nb_treat_all, color=color, linestyle='--', lw=1.5, label=f'_Treat All {idx}')

    # 添加参考线
    treat_none = np.zeros_like(thresholds)
    plt.plot(thresholds, treat_none, color='black', linestyle=':', label='Treat None')
    
    plt.xlabel('Threshold Probability', fontsize=15)
    plt.ylabel('Net Benefit', fontsize=15)
    plt.title(title, fontsize=18)
    
    # 优化图例显示（避免重复标签）
    handles, labels = plt.gca().get_legend_handles_labels()
    filtered_labels = []
    seen = set()
    for h, l in zip(handles, labels):
        if "_Treat All" not in l and l not in seen:
            seen.add(l)
            filtered_labels.append((h, l))
        elif "_Treat All" in l and l not in seen:
            seen.add(l)
            filtered_labels.append((h, l.replace("_Treat All", "Treat All")))
    
    plt.legend(*zip(*filtered_labels), fontsize=10, loc='lower left')
    plt.grid(linestyle='--', alpha=0.7)
    plt.savefig(save_path, format='pdf', bbox_inches='tight')
    plt.show()

In [ ]:
import shap
explainer = shap.TreeExplainer(best_model)
# 计算shap值为numpy.array数组
shap_values_numpy = explainer.shap_values(X)
# 计算shap值为Explanation格式
shap_values_Explanation = explainer(X)

## 针对shap值为numpy.array数组绘制特征贡献图

In [ ]:
# 提取每个类别的 SHAP 值
shap_values_class_0 = shap_values_numpy[:, :, 0]
shap_values_class_1 = shap_values_numpy[:, :, 1]
shap_values_class_2 = shap_values_numpy[:, :, 2]

In [ ]:
# 计算每个类别的特征贡献度
importance_class_0 = np.abs(shap_values_class_0).mean(axis=0)
importance_class_1 = np.abs(shap_values_class_1).mean(axis=0)
importance_class_2 = np.abs(shap_values_class_2).mean(axis=0)

In [ ]:
# 创建一个包含类别特征重要性的 DataFrame
importance_df = pd.DataFrame({
    'Class 0': importance_class_0,
    'Class 1': importance_class_1,
    'Class 2': importance_class_2,
}, index=X_train.columns)

# 根据类别映射表将列名修改为英文描述
type_mapping = {
    0: 'RL0',  # 异常
    1: 'RL1',     # 正常
    2: 'RL2', # 可疑
}

# 修改列名
importance_df.columns = [type_mapping[int(col.split('Class ')[1])] for col in importance_df.columns]
importance_df

In [ ]:
# 添加一列用于存储行的和
importance_df['row_sum'] = importance_df.sum(axis=1)

# 按照行和对DataFrame进行排序
sorted_importance_df = importance_df.sort_values(by='row_sum', ascending=True)

# 删除用于排序的行和列
sorted_importance_df = sorted_importance_df.drop(columns=['row_sum'])

elements = sorted_importance_df.index

# 使用 Seaborn 的颜色调色板，设置为 Set2，以获得对比度更高的颜色
colors = sns.color_palette("Set2_r", n_colors=len(sorted_importance_df.columns))

# 创建图形和坐标轴对象，设置图形大小为12x6英寸，分辨率为1200 DPI
fig, ax = plt.subplots(figsize=(12, 6), dpi=1200)

# 初始化一个数组，用于记录每个条形图的底部位置，初始为0
bottom = np.zeros(len(elements))

# 遍历每个类别并绘制水平条形图
for i, column in enumerate(sorted_importance_df.columns):
    ax.barh(
        sorted_importance_df.index,     # y轴的特征名称
        sorted_importance_df[column],  # 当前类别的SHAP值
        left=bottom,                   # 设置条形图的起始位置
        color=colors[i],               # 使用调色板中的颜色
        label=column                   # 为图例添加类别名称
    )
    # 更新底部位置，以便下一个条形图能够正确堆叠
    bottom += sorted_importance_df[column]

# 设置x轴标签和标题
ax.set_xlabel('mean(SHAP value|)(average impact on model output magnitude)', fontsize=12)
ax.set_ylabel('Features', fontsize=12)
ax.set_title('Feature Importance by Class', fontsize=15)
# 设置y轴刻度和标签
ax.set_yticks(np.arange(len(elements)))
ax.set_yticklabels(elements, fontsize=10)
# 在条形图的末尾添加文本标签
for i, el in enumerate(elements):
    ax.text(bottom[i], i, ' ' + str(el), va='center', fontsize=9)
# 添加图例，并设置图例的字体大小和标题
ax.legend(title='Class', fontsize=10, title_fontsize=12)
# 禁用y轴的刻度和标签
ax.set_yticks([])  # 移除y轴刻度
ax.set_yticklabels([])  # 移除y轴刻度标签
ax.set_ylabel('')  # 移除y轴标签
# 移除顶部和右侧的边框，以获得更清晰的图形
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.savefig('average impact on model output magnitude.pdf', format='pdf', bbox_inches='tight')
plt.show()

## 模型单样本解释

In [ ]:
plt.figure(figsize=(10, 5), dpi=1200)
# 绘制第2个样本第0个类别的 SHAP 瀑布图，并设置 show=False 以避免直接显示
shap.plots.waterfall(shap_values_Explanation[:,:,0][1], show=False, max_display=13)
# 保存图像为 PDF 文件
plt.savefig("SHAP_Waterfall_Plot_Sample_1_0类.pdf", format='pdf', bbox_inches='tight')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5), dpi=1200)
# 绘制第2个样本第1个类别的 SHAP 瀑布图，并设置 show=False 以避免直接显示
shap.plots.waterfall(shap_values_Explanation[:,:,1][1], show=False, max_display=13)
# 保存图像为 PDF 文件
plt.savefig("SHAP_Waterfall_Plot_Sample_1_1类.pdf", format='pdf', bbox_inches='tight')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5), dpi=1200)
# 绘制第2个样本第2个类别的 SHAP 瀑布图，并设置 show=False 以避免直接显示
shap.plots.waterfall(shap_values_Explanation[:,:,2][1], show=False, max_display=13)
# 保存图像为 PDF 文件
plt.savefig("SHAP_Waterfall_Plot_Sample_1_2类.pdf", format='pdf', bbox_inches='tight')
plt.tight_layout()
plt.show()

# PDP解释

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

# 假设 best_model 为训练好的多分类模型，X 为用于解释的数据集
features = ['mGGO']  # 替换为你想要绘制的特征
# 指定类别目标，假设目标类别是 0，可以根据需求替换成 1 或 2
target_class = 0
# 绘制特定类别的部分依赖图
PartialDependenceDisplay.from_estimator(
    best_model, 
    X, 
    features, 
    kind='average', 
    target=target_class
)
plt.show()

In [ ]:
features = ['mGGO'] 
PartialDependenceDisplay.from_estimator(best_model, X, features, kind='individual', target=0)
plt.show()

In [ ]:
# 获取数据集的所有特征名称
features = X.columns

# 创建一个 3x6 的子图布局
fig, axes = plt.subplots(3, 4, figsize=(15, 10), dpi=1200)
axes = axes.ravel()  # 将子图矩阵展平，方便逐个操作

# 针对类别0绘制每个特征的ICE图
for i, feature in enumerate(features):
    try:
        PartialDependenceDisplay.from_estimator(
            best_model, 
            X, 
            [feature], 
            kind='individual', 
            target=0, 
            ax=axes[i]  # 将图绘制到相应的子图中
        )
        axes[i].set_title(f'ICE for {feature}', fontsize=10)
    except Exception as e:
        print(f"Error plotting feature {feature}: {e}")
        axes[i].text(0.5, 0.5, 'Error', fontsize=15, ha='center')
        axes[i].set_title(f'Error for {feature}', fontsize=10)

# 调整布局，防止子图之间重叠
plt.tight_layout()
plt.savefig('ICE_plots_0.pdf', format='pdf', bbox_inches='tight',dpi=1200)
plt.show()

In [ ]:
# 创建一个 3x6 的子图布局
fig, axes = plt.subplots(3, 4, figsize=(15, 10), dpi=1200)
axes = axes.ravel()  # 将子图矩阵展平，方便逐个操作

# 针对类别1绘制每个特征的ICE图
for i, feature in enumerate(features):
    try:
        PartialDependenceDisplay.from_estimator(
            best_model, 
            X, 
            [feature], 
            kind='individual', 
            target=1, 
            ax=axes[i]  # 将图绘制到相应的子图中
        )
        axes[i].set_title(f'ICE for {feature}', fontsize=10)
    except Exception as e:
        print(f"Error plotting feature {feature}: {e}")
        axes[i].text(0.5, 0.5, 'Error', fontsize=15, ha='center')
        axes[i].set_title(f'Error for {feature}', fontsize=10)

# 调整布局，防止子图之间重叠
plt.tight_layout()
plt.savefig('ICE_plots_1.pdf', format='pdf', bbox_inches='tight',dpi=1200)
plt.show()

In [ ]:
# 创建一个 3x6 的子图布局
fig, axes = plt.subplots(3, 4, figsize=(15, 10), dpi=1200)
axes = axes.ravel()  # 将子图矩阵展平，方便逐个操作

# 针对类别2绘制每个特征的ICE图
for i, feature in enumerate(features):
    try:
        PartialDependenceDisplay.from_estimator(
            best_model, 
            X, 
            [feature], 
            kind='individual', 
            target=2, 
            ax=axes[i]  # 将图绘制到相应的子图中
        )
        axes[i].set_title(f'ICE for {feature}', fontsize=10)
    except Exception as e:
        print(f"Error plotting feature {feature}: {e}")
        axes[i].text(0.5, 0.5, 'Error', fontsize=15, ha='center')
        axes[i].set_title(f'Error for {feature}', fontsize=10)

# 调整布局，防止子图之间重叠
plt.tight_layout()
plt.savefig('ICE_plots_2.pdf', format='pdf', bbox_inches='tight',dpi=1200)
plt.show()

In [ ]:
# 选择两个特征绘制2D PDP
features = ['INTER_PERI_0-10mm', 'DL-152-score']  

# 使用 contour_kw 参数绘制2D PDP
fig, ax = plt.subplots(figsize=(10, 6))
PartialDependenceDisplay.from_estimator(
    best_model,
    X,
    features=[features],
    kind='average',
    target=0,
    grid_resolution=50,
    contour_kw={'cmap': 'viridis', 'alpha': 0.8},
    ax=ax
)
plt.suptitle('2D Partial Dependence Plot for INTER_PERI_0-10mm and DL-152-score')
plt.savefig('2D Partial Dependence Plot for INTER_PERI_0-10mm and DL-152-score.pdf', format='pdf', bbox_inches='tight',dpi=1200)
plt.show()

In [ ]:
import joblib
# 保存模型
joblib.dump(best_model , 'XGBoost.pkl')